In [570]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [571]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

X = mnist.data.astype('float32')/255.0
y = mnist.target.astype('int')

In [572]:
X = pd.DataFrame(X)
X['Label'] = y
X

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,...,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.988235,0.164706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [573]:
data = np.array(X)
m, n = data.shape
np.random.shuffle(data)

data_test = data[0:10000].T
X_test = data_test[0:n-1]
y_test = data_test[n-1]

data_train = data[10000:m].T
X_train = data_train[0:n-1]
y_train = data_train[n-1]

In [574]:
y_train = y_train.astype(int)
y_train

array([8, 5, 9, ..., 5, 1, 0])

In [575]:
def init_param():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

In [576]:
def ReLU(x):
    return np.maximum(0, x)

def softMax(x):
    exp_x = np.exp(x)
    return exp_x/sum(np.exp(x))

In [577]:
def fwd_propogation(X_train, W1, b1, W2, b2):
    # First Layer
    Z1 = W1.dot(X_train) + b1
    A1 = ReLU(Z1)

    # Second Layer
    Z2 = W2.dot(A1) + b2
    A2 = softMax(Z2)
    return Z1, A1, Z2, A2

In [578]:
def one_hot(y_train):
    one_hot = np.zeros((y_train.size, (y_train.max()) + 1))
    one_hot[np.arange(y_train.size), y_train] = 1
    one_hot = one_hot.T
    return one_hot

In [579]:
def derivative_ReLU(Z):
    return Z > 0

In [580]:
def back_Propogation(Z1, A1, Z2, A2, W2, X_train, y_train):
    Y = one_hot(y_train)

    #Back Propogation to Hidden Layer(1st Layer) -> 0 based indexing of layers.
    dz2 = A2 - Y
    dw2 = (1.0/Y.size)*dz2.dot(A1.T)
    db2 = (1.0/Y.size)*np.sum(dz2)

    #Back Propogation to Input Layer(0th Layer) -> 0 based indexing of layers.
    dz1 = W2.T.dot(dz2)*derivative_ReLU(Z1)
    dw1 = (1.0/Y.size)*dz1.dot(X_train.T)
    db1 = (1.0/Y.size)*np.sum(dz1)

    return dw1, db1, dw2, db2

In [581]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y)/y.size

In [582]:
def update_params(W1, b1, W2, b2, dw1, db1, dw2, db2, alpha):
    W1 = W1 - alpha * dw1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dw2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [583]:
def gradient_descent(X, y, iterations, alpha):
    W1, b1, W2, b2 = init_param()
    for i in range(0, iterations):
        Z1, A1, Z2, A2 = fwd_propogation(X, W1, b1, W2, b2)
        dw1, db1, dw2, db2 = back_Propogation(Z1, A1, Z2, A2, W2, X, y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dw1, db1, dw2, db2, alpha)
        if i % 10 == 0:
            print(f"Iteration: {i}")
            print(f"Accuracy: {get_accuracy(get_predictions(A2), y)*100.0}")
    return W1, b1, W2, b2

In [584]:
W1, b1, W2, b2 = gradient_descent(X_train, y_train, 100, 0.10)

Iteration: 0
[6 6 6 ... 9 6 6] [8 5 9 ... 5 1 0]
Accuracy: 9.321428571428571
Iteration: 10
[6 6 3 ... 3 6 6] [8 5 9 ... 5 1 0]
Accuracy: 11.138571428571428
Iteration: 20
[6 6 3 ... 3 6 6] [8 5 9 ... 5 1 0]
Accuracy: 12.631428571428572
Iteration: 30
[6 8 3 ... 3 6 5] [8 5 9 ... 5 1 0]
Accuracy: 13.965714285714286
Iteration: 40
[6 8 3 ... 3 6 5] [8 5 9 ... 5 1 0]
Accuracy: 15.057142857142857
Iteration: 50
[6 8 3 ... 3 6 5] [8 5 9 ... 5 1 0]
Accuracy: 15.857142857142856
Iteration: 60
[6 8 3 ... 3 6 5] [8 5 9 ... 5 1 0]
Accuracy: 16.708571428571428
Iteration: 70
[6 8 3 ... 3 6 5] [8 5 9 ... 5 1 0]
Accuracy: 17.45
Iteration: 80
[6 8 3 ... 3 6 5] [8 5 9 ... 5 1 0]
Accuracy: 18.11
Iteration: 90
[6 8 3 ... 3 6 5] [8 5 9 ... 5 1 0]
Accuracy: 18.847142857142856


In [587]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = fwd_propogation(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    # Extract the current image and reshape to (784, 1) for a single image
    current_image = X_train[:, index, None]
    
    # Make prediction for the current image
    prediction = make_predictions(current_image, W1, b1, W2, b2)
    label = y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    # Reshape the image to (28, 28) for visualization and multiply by 255 for proper display
    current_image = current_image.reshape((28, 28)) * 255
    current_image = np.clip(current_image, 0, 255)  # Ensure values are in the range [0, 255]
    
    # Plot the image
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()


In [588]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

ValueError: shapes (10,1) and (10,784) not aligned: 1 (dim 1) != 10 (dim 0)